In [1]:
from dotenv import load_dotenv
import os
import msal
import requests
import json

NUMBER_INDEX = 1

env_path = os.path.join(os.getcwd(), '.env')
load_dotenv(dotenv_path=env_path)

USERS_TABLE_URL = f"https://graph.microsoft.com/v1.0/drives/{os.environ.get('DRIVE_ID')}/items/{os.environ.get('USERS_FILE_ID')}/workbook/worksheets/Users/tables/UsersTable/rows"
LOOKUP_TABLE_URL = f"https://graph.microsoft.com/v1.0/drives/{os.environ.get('DRIVE_ID')}/items/{os.environ.get('USERS_FILE_ID')}/workbook/worksheets/Lookup/tables/LookupTable/rows"

# Enter details of AAD app registration

config = {
    'client_id': os.environ.get('CLIENT_ID'),
    'client_secret': os.environ.get('CLIENT_SECRET'),
    'authority': os.environ.get('AUTHORITY'),
    'scope': [os.environ.get('SCOPE')],
    'site_id': os.environ.get('SITE_ID'),
}


def get_users_info( msal_instance, scope):
    '''Returns a 2D list containing the user details within the inner array'''

    # First, try to lookup an access token in cache
    token_result = msal_instance.acquire_token_silent(scope, account=None)

    # If the token is available in cache, save it to a variable
    if token_result:
        print('Access token was loaded from cache')

    # If the token is not available in cache, acquire a new one from Azure AD and save it to a variable
    if not token_result:
        print(scope)
        token_result = msal_instance.acquire_token_for_client(scopes=scope)
        print(token_result)
        access_token = 'Bearer ' + token_result['access_token']
        print(f'New access token {access_token} was acquired from Azure AD')


    # Copy access_toek and specify the MS Graph API endpoint you want to call, e.g. '
    headers = {
        'Authorization': access_token
    }

    # Make a GET request to the provided url, passing the access token in a header
    users_request = requests.get(url=USERS_TABLE_URL, headers=headers)
    lookups_request = requests.get(url=LOOKUP_TABLE_URL, headers=headers)

    user_arrs = [tuple(info) for object_info in users_request.json()['value'] for info in object_info['values']]
    lookups_arrs = [tuple(info) for object_info in lookups_request.json()['value'] for info in object_info['values']]

    print(lookups_arrs)



    # user_info = [info for object_info in graph_result.json()['value'] for info in object_info['values'] if int(info[NUMBER_INDEX]) == from_number]

    print(f"user info: {user_arrs}")

    return (user_arrs, lookups_arrs) # info is already a list so user_info is a 2D list


# create an MSAL instance providing the client_id, authority and client_credential params
def get_msal_instance():
    return msal.ConfidentialClientApplication(config['client_id'], authority=config['authority'], client_credential=config['client_secret'])

msal_instance = get_msal_instance()
tables = get_users_info(msal_instance, config['scope'])

['https://graph.microsoft.com/.default']
{'token_type': 'Bearer', 'expires_in': 3599, 'ext_expires_in': 3599, 'access_token': 'eyJ0eXAiOiJKV1QiLCJub25jZSI6IjZzY2ZoS2t3cUlxRWgzdTQ2RmxSa3E0MlFWNTNUUVRRb1NwZFlxZ25VSDgiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC9jODE5ZjQyZC02ODhmLTRjODUtYmJjNi0xY2EwNWZkMjMzMGEvIiwiaWF0IjoxNjk3MDgzNDYwLCJuYmYiOjE2OTcwODM0NjAsImV4cCI6MTY5NzA4NzM2MCwiYWlvIjoiRTJGZ1lKZ200WDdHOHRucUQxdTJQTEgrZE9MeFB3QT0iLCJhcHBfZGlzcGxheW5hbWUiOiJRdWVyeSBNUyBHcmFwaCIsImFwcGlkIjoiNmRlMzk4ZDQtODM5NC00MTI0LWIxYmUtNWFhYmUxNGE3ZDFmIiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvYzgxOWY0MmQtNjg4Zi00Yzg1LWJiYzYtMWNhMDVmZDIzMzBhLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiZGYzOTEyYWQtZTNhZi00MWZhLTk4ZjYtMGViODdhMzZiYzYwIiwicmgiOiIwLkFYSUFMZlFaeUk5b2hVeTd4aHlnWDlJekNnTUFBQUFBQUFBQXdBQUFBQUFBQUFCeUFBQS4iLCJyb2xlcyI6WyJTaXRl

In [2]:
import numpy as np
import pandas as pd
users = pd.DataFrame(data=tables[0], columns=["name", "number", "email"])

def df_replace_spaces(df):
    df.replace('', np.nan, inplace=True)
    df = df.dropna(how="all", inplace=True)
    return df

df_replace_spaces(users)
users['number'] = users["number"].astype(int)


users

,name,number,email
0,Rachmiel Teo Ren Xiang,88584969,rach@go.edu.sg
1,Shawn Tan Minyi,91681054,shawn@go.edu.sg
2,Tymothy LimJie,84885787,tymothy@go.edu.sg
3,Lim Zong Han,88252235,zonghan@go.edu.sg


In [3]:
lookups = pd.DataFrame(data = tables[1], columns=["name", "reporting_officer_name", "hod_name"])

df_replace_spaces(lookups)

lookups

,name,reporting_officer_name,hod_name
0,Rachmiel Teo Ren Xiang,Lim Zong Han,Lim Zong Han
1,Shawn Tan Minyi,Rachmiel Teo Ren Xiang,Rachmiel Teo Ren Xiang
2,Tymothy LimJie,Rachmiel Teo Ren Xiang,Lim Zong Han


In [4]:
from flask import Flask, request, jsonify
import sqlite3

from datetime import datetime
from sqlalchemy.orm import Mapped, mapped_column
import os
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import event

basedir = os.path.join(os.getcwd())

class Config:

    # Database
    if not os.getenv("DATABASE_URL") is None:
        SQLALCHEMY_DATABASE_URI = os.getenv("DATABASE_URL")
        STATIC_FOLDER = os.path.join(os.getenv("APP_FOLDER"), "project", "static")
        UPLOADS_FOLDER = os.path.join(os.getenv("APP_FOLDER"), "project", "uploads")
    else:
        SQLALCHEMY_DATABASE_URI = 'sqlite:///' + os.path.join(basedir, 'chatbot.db')
        SQLALCHEMY_TRACK_MODIFICATIONS = False

app = Flask(__name__)
db = SQLAlchemy()
app.config.from_object(Config)
db.init_app(app)

class User(db.Model):

    __tablename__ = "user"
    name: Mapped[str] = mapped_column(db.String(80), primary_key=True, nullable=False)
    number: Mapped[str] = mapped_column(db.Integer(), unique=True, nullable=False)
    email: Mapped[str] = mapped_column(db.String(120), unique=True, nullable=False)
    reporting_officer: Mapped[str] = mapped_column(db.String(50))
    hod: Mapped[str] = mapped_column(db.String(50))

    # Self-referential relationships
    reporting_officer_name: Mapped[str] = mapped_column(db.String(80), db.ForeignKey('user.name', ondelete="CASCADE"), nullable=True)
    reporting_officer = db.relationship('User', backref=db.backref('subordinates', cascade='all, delete-orphan'), remote_side=[name], post_update=True, foreign_keys=[reporting_officer_name])
    
    hod_name: Mapped[str] = mapped_column(db.String(80), db.ForeignKey('user.name', ondelete="CASCADE"), nullable=True)
    hod = db.relationship('User', backref=db.backref('dept_members', cascade='all, delete-orphan'), remote_side=[name], post_update=True, foreign_keys=[hod_name])

    users = db.relationship('McDetails', backref="user", lazy=True)

    def __init__(self, name, number, email):
        self.name = name
        self.number = number
        self.email = email

class McDetails(db.Model):

    __tablename__ = "mc_details"
    id: Mapped[str] = mapped_column(db.String(50), nullable=False, primary_key=True)
    number: Mapped[str] = mapped_column(db.Integer(), nullable=False)
    name: Mapped[str] = mapped_column(db.String(50), db.ForeignKey('user.name'), nullable=False)
    start_date: Mapped[str] = mapped_column(db.String(20), nullable=False)
    end_date: Mapped[str] = mapped_column(db.String(20), nullable=False)
    duration: Mapped[str] = mapped_column(db.Integer, nullable=False)
    intent: Mapped[int] = mapped_column(db.Integer(), nullable=False)
    status: Mapped[int] = mapped_column(db.Integer(), nullable=False)
    timestamp: Mapped[datetime] = mapped_column(db.DateTime, default=datetime.utcnow)

    def __init__(self, id, number, name, start_date, end_date, duration, intent, status, timestamp=datetime.utcnow):
        self.id = id
        self.number = number
        self.name = name
        self.start_date = start_date
        self.end_date = end_date
        self.duration = duration
        self.intent = intent
        self.status = status
        self.timestamp = timestamp

def create_db():
    with app.app_context():
        db.create_all()
        db.session.commit()


def remove_db():
    with app.app_context():
        db.drop_all()
        db.session.commit()

def seed_db():
    user = User("Rachmiel", "12345678", "rach@rach")
    db.session.add(user)
    db.session.commit()

create_db()




In [5]:
with app.app_context():
    seed_db()

In [5]:
conn = sqlite3.connect('chatbot.db')
cursor = conn.cursor()
cursor.execute('''SELECT * FROM user''')
row = cursor.fetchall()
conn.commit()
conn.close()

print(row)

[]


In [6]:
# hod_df = users[users.index.isin(lookups["hod"])].copy()
# df_replace_spaces(hod_df)

#SECTION start to update df
az_users = users.merge(lookups, how="outer", left_on="name", right_on="name", indicator=True)
az_users = az_users[az_users._merge != "right_only"].drop(columns="_merge")
az_users.sort_values(by="name", inplace=True)

col_order = ['name', 'number', 'email', 'reporting_officer_name', 'hod_name']
az_users = az_users[col_order]

az_users

,name,number,email,reporting_officer_name,hod_name
3,Lim Zong Han,88252235,zonghan@go.edu.sg,NaN,NaN
0,Rachmiel Teo Ren Xiang,88584969,rach@go.edu.sg,Lim Zong Han,Lim Zong Han
1,Shawn Tan Minyi,91681054,shawn@go.edu.sg,Rachmiel Teo Ren Xiang,Rachmiel Teo Ren Xiang
2,Tymothy LimJie,84885787,tymothy@go.edu.sg,Rachmiel Teo Ren Xiang,Lim Zong Han


In [7]:
conn = sqlite3.connect('chatbot.db')
cursor = conn.cursor()
cursor.execute('''SELECT * FROM user ORDER BY name ASC''')
db_users = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

conn.commit()
conn.close()

db_users = pd.DataFrame(db_users, columns=column_names)
db_users.sort_index(axis=1, inplace=True)
db_users = db_users[col_order]
db_users

,name,number,email,reporting_officer_name,hod_name


In [8]:
import traceback

exact_match = az_users.equals(db_users)
print(exact_match)

if not exact_match:
    # create 2 dataframes to compare
    old_users = pd.merge(az_users, db_users, how="outer", indicator=True).query('_merge == "right_only"').drop(columns='_merge')
    new_users = pd.merge(az_users, db_users, how="outer", indicator=True).query('_merge == "left_only"').drop(columns='_merge')

    old_users_tuples = [tuple(old_user) for old_user in old_users.values]
    new_users_tuples = [tuple(new_user) for new_user in new_users.values]
    print(old_users_tuples)
    print(new_users_tuples)


    conn = sqlite3.connect('chatbot.db')
    cursor = conn.cursor()

    for name, number, email, reporting_officer, hod in old_users_tuples:
        cursor.execute('DELETE FROM user WHERE name = ?', (name, ))

    conn.commit()
    conn.close()

    conn = sqlite3.connect('chatbot.db')
    cursor = conn.cursor()
        
    for name, number, email, reporting_officer, hod in new_users_tuples:
        cursor.execute('INSERT INTO user (name, number, email) VALUES (?, ?, ?)', (name, number, email))

    conn.commit()
    conn.close()

    conn = sqlite3.connect('chatbot.db')
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys=ON")
        
    for name, number, email, reporting_officer, hod in new_users_tuples:
        cursor.execute('UPDATE user SET reporting_officer_name = ?, hod_name = ? WHERE name = ?', (reporting_officer, hod, name))

    conn.commit()
    conn.close()
    
   


False
[]
[('Lim Zong Han', 88252235, 'zonghan@go.edu.sg', nan, nan), ('Rachmiel Teo Ren Xiang', 88584969, 'rach@go.edu.sg', 'Lim Zong Han', 'Lim Zong Han'), ('Shawn Tan Minyi', 91681054, 'shawn@go.edu.sg', 'Rachmiel Teo Ren Xiang', 'Rachmiel Teo Ren Xiang'), ('Tymothy LimJie', 84885787, 'tymothy@go.edu.sg', 'Rachmiel Teo Ren Xiang', 'Lim Zong Han')]


In [9]:
az_users.loc[2, "reporting_officer_name"] = 'Shawn Tan Minyi'

In [10]:
az_users.loc[2, "reporting_officer_name"]

'Shawn Tan Minyi'

In [11]:
with app.app_context():
    first_user = User.query.first()
    hod = first_user.hod


print(first_user)
print(first_user.number)
print(hod)

<User Lim Zong Han>
88252235
None
